Author: Suhan Shetty (suhan.n.shetty@gmail.com | suhan.shetty@idiap.ch)

This notebook implements multilinear PCA based regression for tensor time series modeling

References:

 - Dynamic tensor time series modeling and analysis, https://ieeexplore.ieee.org/abstract/document/7798500

 - MULTILINEAR TENSOR REGRESSION FOR LONGITUDINAL RELATIONAL DATA:
 https://www.jstor.org/stable/43826417
 
 - Multilinear Dynamical Systems for Tensor Time Series: https://people.eecs.berkeley.edu/~russell/papers/nips13-tensor.pdf


In [1]:
import numpy as np
import tensorly as tl
from tensorly.tenalg import multi_mode_dot
from tensorly.base import unfold
from scipy.sparse.linalg import eigsh
from scipy.linalg import pinv
from tensorly.tenalg import kronecker as kron


In [2]:
N = 3 # dimension of each output tensor Yi's
T = 100 # length of time series
shape_Yi = [32,32,5]
I = [T,*shape_Yi]

# Generate data Y from some low-rank structure 
J0 = [15,20,3] # ranks of the factor matrices
U0 = [np.random.rand(I[n+1],J0[n]) for n in range(N)] #Factors
X0 = np.random.rand(T,*J0) # Core array

Y = multi_mode_dot(X0,U0, modes=[n+1 for n in range(N)]) #Output tensor
    

In [3]:
Y_mean = np.average(Y,axis=0)
Yn = Y - Y_mean # mean zero

In [4]:
def mpca(Y,J,tol=1e-2):
    '''
    Y: an N-dimensional Tensor
    J: Desired ranks of the tucker factors (increase J for better approximation)
    tol: tolerance for convergence of mpca iterations
    '''
    Yn = Y-np.average(Y,axis=0)
    # Initialization
    U = [None]*(len(Yn.shape)-1)
    for n in range(N): # across all modes
        mode_n_sum = 0
        for t in range(T):
            unfold_along_n = unfold(Yn[t],mode=n)
            mode_n_sum += unfold_along_n@unfold_along_n.T
        U[n] = eigsh(mode_n_sum,k=J[n])[1] # faster way to compute eigen vectors (for symmetric matrices)
        # Note: The most significant eigen vectors are arranged in the last columns of U
    
    # Local optimization
    X = multi_mode_dot(Yn,U,modes=[1+i for i in range(N)], transpose=True)
    Yn_apprx0 = multi_mode_dot(X,U,modes=[1+i for i in range(N)])
    convergence = False
    for m in range(500): #iterate until convergence
        for n in range(N):
            U_ = U[:]
            U_.pop(n)
            mode_n_sum = 0
            U_kron = kron(U_,reverse=True)
            for t in range(T):
                unfold_along_n = unfold(Yn[t],mode=n)
                mode_n_sum += unfold_along_n@U_kron@U_kron.T@unfold_along_n.T
            U[n] = eigsh(mode_n_sum,k=J[n])[1]
        
        X = multi_mode_dot(Yn,U,modes=[1+i for i in range(N)], transpose=True) # Current cores
        
        # Check convergence
        Yn_apprx = multi_mode_dot(X,U,modes=[1+i for i in range(N)])#Curent approximation
        if np.linalg.norm(Yn_apprx-Yn_apprx0) < tol:
            print("mpca converged at iteration: ",m)
            convergence = True
            break
        else:
            Yn_apprx0 = 1*Yn_apprx
    
    if convergence==False:
        print("mpca has not yet converged")
        
    X = multi_mode_dot(Y,U,modes=[1+i for i in range(N)], transpose=True)
    Y_apprx = multi_mode_dot(X,U,modes=[1+i for i in range(N)])
    print("Error in approximation: ", np.linalg.norm(Y-Y_apprx)/np.linalg.norm(Y))

    return X,U

In [5]:
X,U = mpca(Y, J=J0) 

mpca converged at iteration:  0
Error in approximation:  8.983225110648771e-16


In [6]:
# Multilinear Regression
# Ref:MULTILINEAR TENSOR REGRESSION FOR LONGITUDINAL RELATIONAL DATA:
# https://www.jstor.org/stable/43826417?seq=1#metadata_info_tab_contents
def multilinear_regression(X,Y,tol=0.0001):
    '''
    X: Tx?x?x..x? : {X_i}, i=1,..,T
    Y: Tx?x?x..x?: {Y_i}, i=1,...,T 
    N: dimension of the tensor X_i (or Y_i)
    J: ranks of the factor matrices
    return the model: Y_i = X_ix{U_1,..,U_N}
    '''
    I = Y.shape
    J = X.shape
    U = [100*np.random.rand(I[n+1],J[n+1]) for n in range(N)] 
    #Note: Fix the initialization. Somehow, smaller values of initialization, results in slower convergence.
    # Understand this problem
    U.insert(0,np.eye(I[0])) 
    Y_apprx0 = multi_mode_dot(X,U,modes=[i for i in range(N+1)])
    convergence = False
    for m in range(1000):
        for n in range(N):
            U_ = U[:]
            U_[n+1] = np.eye(J[n+1])
            X_tmp = multi_mode_dot(X,U_,modes=[i for i in range(N+1)]) 
            X_tmp_n = unfold(X_tmp,mode=n+1)
            Y_n = unfold(Y,mode=n+1)
            U[n+1] = Y_n@pinv(X_tmp_n)
            if n<(N-1):#to make sure the values dont blow too much
                U[n+1] = U[n+1]/(np.linalg.norm(U[n+1])+1.)
       
        # Check convergence
        Y_apprx = multi_mode_dot(X,U,modes=[i for i in range(N+1)])#Curent approximation
        
        if np.linalg.norm(Y_apprx-Y_apprx0) < tol:
            print("Regression converged at iteration: ",m)
            convergence = True
            break
        else:
            if (m+1)%1==0:
                print("Current error in approximation:", np.linalg.norm(Y-Y_apprx)/np.linalg.norm(Y))
            Y_apprx0 = 1*Y_apprx

    
    if convergence==False:
        print("Multilinear regression has not yet converged")
    
    print("Final error in approximation:", np.linalg.norm(Y-Y_apprx)/np.linalg.norm(Y))
    
    return U[1:]


In [7]:
U = multilinear_regression(X,Y,tol=0.01)

Current error in approximation: 0.07689088135692322
Current error in approximation: 0.004375508413595701
Current error in approximation: 0.0006952831545306575
Current error in approximation: 0.0001388050684602708
Current error in approximation: 3.266251127994519e-05
Current error in approximation: 8.12908517718187e-06
Current error in approximation: 2.06988042732246e-06
Current error in approximation: 5.343555608891442e-07
Current error in approximation: 1.3942544256873205e-07
Regression converged at iteration:  9
Final error in approximation: 3.6715126122381255e-08


In [8]:
Y.shape

(100, 32, 32, 5)

In [9]:
# Compute Array-normal distribution 

# Compute Matrix root--------------------------------------------------------------------------------------
# Compute A's from Cov: Cov = U*S*Vh, A = U*Sqrt(S)
def mat_root(Cov,tol=1e-9):
    U,s,Vh = np.linalg.svd(Cov)
    idx = s>tol
    s = s[idx]
    U = U[:,idx]   
    s_r = np.sqrt(s)
    S_r = np.diag(s_r)
    S_inv_r = np.diag(1/s_r)
    A = U@S_r
    A_inv = S_inv_r@U.T
    return (A, A_inv) # retun both the sqrt and its inverse

# Separable Covariance estimation for N-way array data using MLE
def array_normal(X, tol=1e-2): 
    '''
    Data points {Xi}, i=1,..,T are arranged into one array X
    Input tensor X. Xi = X[i], i =1,..,T are the data points
    '''    
    I = X.shape 
    T = X.shape[0]
    N  = len(I)-1 # dimension of the data points X_i
    
    # Compute the mean:
    Xavg = np.average(X,axis=0)
    
    Xe = X - Xavg

    A = [100*np.random.rand(I[i+1],I[i+1]) for i in range(N)]
    Cov = [A_@A_.T for A_ in A]   
    
    # X[i] ~ Xavg + Z x {Cov1, Cov2,..., CovN}, and CovK = Ak*Ak'
    A_inv = [None]*T
    for n in range(N):
        (A[n],A_inv[n]) = mat_root(Cov[n])
    A_inv.insert(0,np.identity(T))
    A.insert(0,np.identity(T))
    Cov.insert(0,np.identity(T))
    
    for reps in range(100):
        for n in range(N): #iterate over each mode
            A_inv_ = A_inv[:]
            A_ = A[:]
            Cov_ = Cov[:]
            A_[n+1] = np.eye(I[n+1])
            A_inv_[n+1] = np.eye(I[n+1])
            Xe_tmp = multi_mode_dot(Xe,A_inv_,modes=[i for i in range(N+1)])
            Xe_tmp_n = unfold(Xe_tmp,mode=n+1)
            c = T*np.prod(I[1:])/I[n+1]
            Cov[n+1] = (Xe_tmp_n@Xe_tmp_n.T)/c
            if n<(N-1):#to make sure the values dont blow too much
                Cov[n+1] = Cov[n+1]/(np.linalg.norm(Cov[n+1])+1.)
                
            (A[n+1],A_inv[n+1]) = mat_root(Cov[n+1])
         
        err = np.linalg.norm(kron(Cov[1:])-kron(Cov_[1:]))
        if err<tol:
            print("MLE for array normal converged in ", reps, " steps")
            break
            
    return (Xavg,Cov[1:],A[1:], A_inv[1:])

# Sampling from array-normal distribution---------------------------------------------------------------------------
def anormal_sampling(Xavg,Cov):
    N = len(Xavg.shape)
    # Compute the matrix-square-root (Cov = A*A') of covariance matrices
    A = [mat_root(Cov[n])[0] for n in range(N)]
    Z = np.random.randn(*Xavg.shape)
    X = Xavg + multi_mode_dot(Z,A,modes=[n for n in range(N)])
    return X


In [10]:
X0 = np.random.randn(5,6,7,8) # test data

In [11]:
Xavg, Cov, _, _= array_normal(X0, tol=0.01) # Fit an array-normal distribution

In [12]:
Xsample = anormal_sampling(Xavg,Cov) # sample from the modeled array-normal distribution

In [13]:
# Multilinear Regression with Array-normal distribution
# Ref:MULTILINEAR TENSOR REGRESSION FOR LONGITUDINAL RELATIONAL DATA:
# https://www.jstor.org/stable/43826417?seq=1#metadata_info_tab_contents
def anormal_regression(X,Y,tol=0.001):
    '''
    X: Tx?x?x..x? : {X_i}, i=1,..,T
    Y: Tx?x?x..x?: {Y_i}, i=1,...,T 
    N: dimension of the tensor X_i (or Y_i)
    J: ranks of the factor matrices
    return the model: Y_i = X_ix{U_1,..,U_N} + N(0,Cov)
    '''
    I = Y.shape
    J = X.shape
    T = X.shape[0]
    # Initialization of factor matrices
    U = [100*np.random.rand(I[n+1],J[n+1]) for n in range(N)] 
    #Note: Fix the initialization. Somehow, smaller values of initialization, results in slower convergence.
    # Understand this problem
    U.insert(0,np.eye(I[0])) 
    Y_apprx0 = multi_mode_dot(X,U,modes=[i for i in range(N+1)]) # current output approximation
    print("Shape of Y_apprx0: ",Y_apprx0.shape)
    # Initialize covariance matrices
    A = [100*np.random.rand(I[i+1],I[i+1]) for i in range(N)]
    Cov = [A_@A_.T for A_ in A]   
    A_inv = [None]*N
    for n in range(N):
        (A[n],A_inv[n]) = mat_root(Cov[n])
    A_inv.insert(0,np.identity(T))
    A.insert(0,np.identity(T))
    Cov.insert(0,np.identity(T))
    convergence = False
    for m in range(1000):
        # With the current estimation of Covariance (i.e A and A_inv) find the factor matrices U
        for reps in range(1):
            for n in range(N):
                U_ = U[:]
                U_[n+1] = np.eye(J[n+1])
                A_inv_ = A_inv[:]
                A_inv_.pop(n+1)
                X_tmp = multi_mode_dot(X,U_,modes=[i for i in range(N+1)]) 
                modes=[i+1 for i in range(N)]
                modes.pop(n)
                X_tmp_rescaled = multi_mode_dot(X_tmp, A_inv_[1:],modes=modes)
                X_tmp_rescaled_n = unfold(X_tmp_rescaled,mode=n+1)
                Y_rescaled = multi_mode_dot(Y, A_inv_[1:],modes=modes)
                Y_rescaled_n = unfold(Y_rescaled,mode=n+1)
                U[n+1] = Y_rescaled_n@pinv(X_tmp_rescaled_n)
                if n<(N-1): # to make sure the values dont blow too much
                    U[n+1] = U[n+1]/(np.linalg.norm(U[n+1])+1e-2)

        # With the current estimation of factor matrices find the Covariances
        Xe = Y-multi_mode_dot(X,U,modes=[i for i in range(N+1)]) # Error in approximation
        Xe = Xe-np.average(Xe,axis=0) # subtract the mean
        for reps in range(5):
            for n in range(N):
                A_inv_ = A_inv[:]
                A_ = A[:]
                Cov_ = Cov[:]
                A_[n+1] = np.eye(I[n+1])
                A_inv_[n+1] = np.eye(I[n+1])
                Xe_tmp = multi_mode_dot(Xe,A_inv_[1:],modes=[i+1 for i in range(N)])
                Xe_tmp_n = unfold(Xe_tmp,mode=n+1)
                c = T*np.prod(I[1:])/I[n+1]
                Cov[n+1] = (Xe_tmp_n@Xe_tmp_n.T)/c
                if n<(N-1):#to make sure the values dont blow too much
                    Cov[n+1] = Cov[n+1]/(np.linalg.norm(Cov[n+1])+1e-2)
                (A[n+1],A_inv[n+1]) = mat_root(Cov[n+1])

        
        
        # Check convergence
        Y_apprx = multi_mode_dot(X,U,modes=[i for i in range(N+1)])#Curent approximation
        err_cov = np.linalg.norm(kron(Cov[1:])-kron(Cov_[1:]))
        err_reg = np.linalg.norm(Y_apprx-Y_apprx0)/np.linalg.norm(Y_apprx)
        print("Current error in approximation | U:", err_reg, " | Cov: ", err_cov)

        if  err_cov < tol and err_reg < tol:
            print("Regression converged at iteration: ",m)
            convergence = True
            break
        else:
            Y_apprx0 = 1*Y_apprx

    
    if convergence==False:
        print("Multilinear regression has not yet converged")
    print("Final error in approximation of Y = X x {U_1,..,U_N}:", np.linalg.norm(Y-Y_apprx)/np.linalg.norm(Y))
    return (U[1:],Cov[1:],A[1:], A_inv[1:])


In [14]:
U, Cov, A, A_inv = anormal_regression(X,Y,tol=0.01)

Shape of Y_apprx0:  (100, 32, 32, 5)
Current error in approximation | U: 63790288.31963627  | Cov:  7.319982924894561
Current error in approximation | U: 0.7503340812583458  | Cov:  0.0022880231408216444
Current error in approximation | U: 0.033399652181985436  | Cov:  495.5614588313141
Current error in approximation | U: 0.08586984422553834  | Cov:  0.413248948672643
Current error in approximation | U: 0.09006379907389192  | Cov:  4.752395978021957e-06
Current error in approximation | U: 0.00032839369129261984  | Cov:  2.0722034920225093e-08
Regression converged at iteration:  5
Final error in approximation of Y = X x {U_1,..,U_N}: 5.347387675559351e-07
